In [68]:
from psycopg2 import connect as pgconnect
import snowflake.connector
import pandas as pd
import numpy as np
import string

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import scipy.sparse as sparse
import pickle
import csv
import implicit
import itertools
import copy
plt.style.use('ggplot')

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F 
from torch.optim.lr_scheduler import _LRScheduler

#dl libraraies
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau


from keras.layers.merge import dot
from keras.models import Model


# specifically for deeplearning.
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#get recipe date

getrecipes = """
SELECT go_live_at, content_id as mid,title_en, title_de, title_zh
FROM recipe_wagtailrecipe
    join wagtailcore_page p on recipe_wagtailrecipe.page_ptr_id = p.id
"""

con = pgconnect("host='localhost' port='5433' user='bi_team' password='TbuEG§RjC8yNefUJsbuVAWYZU' dbname='ultron'")
recipe_all = pd.read_sql(getrecipes, con) 

In [3]:
recipe_all.head()

,go_live_at,mid,title_en,title_de,title_zh
0,2050-11-11 11:00:00+00:00,R1446,None,Cupcakes mit kinder Riegel®,None
1,2019-02-25 00:00:00+00:00,R1602,Mushroom stroganoff,Pilz-Stroganoff mit Nudeln,俄式酸黄瓜蘑菇烩意面
2,2017-07-19 02:00:00+00:00,H55,How to cut broccoli,Brokkoli geschickt schneiden,如何切西兰花
3,2018-05-10 02:00:00+00:00,C113,Papaya-avocado sandwich,Papavo-Brot,木瓜牛油果三明治
4,2018-02-14 02:00:00+00:00,H314,How to make an ice ring for punch,Dekorativer Eisring für Punch,How to make an ice ring for punch


In [93]:
df_all = pd.read_csv('/Users/xun/Documents/Thesis/ks-3m_O/ks-1y-v2.csv', header=None)

In [94]:
df_all = df_all.sort_values(by = [0, 2])

In [95]:
df_all = df_all.reset_index(drop=True)
df_all.head()

,0,1,2
0,1735967714,R776,2017-10-06 03:29:01.133000000
1,1735967714,R890,2017-10-31 11:51:22.468000000
2,1735967714,C26,2017-10-31 11:57:39.506000000
3,1735967714,R496,2017-10-31 12:01:11.523000000
4,1735967714,R341,2017-10-31 12:02:18.817000000


In [96]:
df = df_all.loc[:, 0:1]

In [97]:
df.columns = ['uid', 'mid']
# df['mid']= df['mid'].str.replace('"', '')

In [98]:
df_o = df

In [99]:
df_o.head()

,uid,mid
0,1735967714,R776
1,1735967714,R890
2,1735967714,C26
3,1735967714,R496
4,1735967714,R341


In [100]:
df_o.shape

(2477082, 2)

In [101]:
len(list(df_o['uid'].unique()))
len(list(df_o['mid'].unique()))

45302

1507

In [102]:
# Create mappings
mid_to_idx = {}
idx_to_mid = {}
for (idx, mid) in enumerate(df.mid.unique().tolist()):
    mid_to_idx[mid] = idx
    idx_to_mid[idx] = mid
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df.uid.unique().tolist()):
    uid_to_idx[uid] = idx
    idx_to_uid[idx] = uid

In [103]:
len(mid_to_idx)

1507

In [104]:
len(uid_to_idx)

45302

In [105]:
uid_to_idx[61903714563]

43090

In [106]:
mapping = pd.DataFrame(idx_to_mid.values(), columns = ['mid'])

In [107]:
mapping['idx'] = mapping.index
# mapping.head()

In [108]:
mapping['mid'] = mapping['mid'].str.replace('"', '')

In [109]:
mapping = mapping.merge(recipe_all, 'left', on = 'mid')

In [110]:
mapping.head()

,mid,idx,go_live_at,title_en,title_de,title_zh
0,R776,0,2017-10-05 02:00:00+00:00,Cinnamon rolls,Zimtschnecken,肉桂卷
1,R890,1,2017-10-31 02:00:00+00:00,Sichuan rice noodle and pork stir-fry,Reisnudel-Stir-Fry mit Hackfleisch,蚂蚁上树
2,C26,2,2017-10-12 02:00:00+00:00,Armenian-style flatbread,Armenisches Fladenbrot,亚美尼亚式烤饼
3,R496,3,2016-10-15 03:00:00+00:00,Creamy pumpkin pasta with sage,Cremige Pasta mit Kürbis und Salbei,鼠尾草南瓜泥意面
4,R341,4,2015-11-05 03:00:00+00:00,Hot chickpea soup,Scharfe Kichererbsensuppe,香辣鹰嘴豆浓汤


In [111]:
mapping = mapping[['idx', 'mid', 'title_en', 'title_de', 'title_zh', 'go_live_at']]

In [112]:
mapping.head()

,idx,mid,title_en,title_de,title_zh,go_live_at
0,0,R776,Cinnamon rolls,Zimtschnecken,肉桂卷,2017-10-05 02:00:00+00:00
1,1,R890,Sichuan rice noodle and pork stir-fry,Reisnudel-Stir-Fry mit Hackfleisch,蚂蚁上树,2017-10-31 02:00:00+00:00
2,2,C26,Armenian-style flatbread,Armenisches Fladenbrot,亚美尼亚式烤饼,2017-10-12 02:00:00+00:00
3,3,R496,Creamy pumpkin pasta with sage,Cremige Pasta mit Kürbis und Salbei,鼠尾草南瓜泥意面,2016-10-15 03:00:00+00:00
4,4,R341,Hot chickpea soup,Scharfe Kichererbsensuppe,香辣鹰嘴豆浓汤,2015-11-05 03:00:00+00:00


In [113]:
def map_ids(row, mapper):
    return mapper[row]

In [114]:

I = df.uid.apply(map_ids, args=[uid_to_idx]).as_matrix() # turn uid to idx
J = df.mid.apply(map_ids, args=[mid_to_idx]).as_matrix() # turn mid to idx
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
df.uid = df.uid.apply(map_ids, args=[uid_to_idx])
df.mid = df.mid.apply(map_ids, args=[mid_to_idx])

In [133]:
df.loc[df['uid']==999][-2:]

,uid,mid
60760,999,32
60761,999,199


In [115]:
I.shape

(2477082,)

In [127]:
# I, J are index columns of sparse matrix of user-item matrix, where the value of coordinate
# (i, j) is 1.
df_ij= pd.DataFrame(data=np.column_stack((I,J)),columns=['uid','mid'])
df_ij

,uid,mid
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
5,0,5
6,0,6
7,0,7
8,0,8
9,0,9


In [117]:
likes.todense().shape

(45302, 1507)

In [126]:
likes.getrow(10000).indices[-10:]

array([ 870,  885,  947, 1058, 1119, 1128, 1223, 1252, 1330, 1348],
      dtype=int32)

In [139]:
def train_test_split(ratings, df, split_count, fraction=None):
    """
    Split recommendation data into train and test sets
    
    Params
    ------
    ratings : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all 
        users are considered.
    """
    # Note: likely not the fastest way to do things below.
    train = ratings.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
#     numpy.random.seed(0)
    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*k, fraction))
            raise
    else:
        user_index = range(train.shape[0])
        
    train = train.tolil()

    for user in user_index:
        test_ratings = df.loc[df['uid']==user][-split_count:]['mid'].values
#         test_ratings = np.random.choice(ratings.getrow(user).indices, 
#                                         size=split_count, 
#                                         replace=False)
        train[user, test_ratings] = 0.
        # These are just 1.0 right now
        test[user, test_ratings] = ratings[user, test_ratings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), user_index

In [140]:
np.random.seed(1234)
train, test, user_index = train_test_split(likes, df, 10, fraction=0.1)

In [141]:
user_index[0:5]

[28807, 43059, 7086, 35572, 37257]

In [142]:
#test set size
train2 = likes.copy().tocoo()
train2.shape[0]*0.1

4530.2

In [143]:
a = np.bincount(train2.row) >= 20
len(a)
b = [x for x in a if x == True]
len(b)
len(a) * 0.1
# there are 11351 users who have engaged with more than 2*10 recipes, which are larger than
# size of 0.2*train = 5160, so just random choose 5160 users from those 11351 users

45302

45302

4530.2

In [144]:
train, test, len(user_index)

(<45302x1507 sparse matrix of type '<class 'numpy.float64'>'
 	with 2431782 stored elements in Compressed Sparse Row format>,
 <45302x1507 sparse matrix of type '<class 'numpy.float64'>'
 	with 45300 stored elements in Compressed Sparse Row format>,
 4530)

In [145]:
user = train.nonzero()[0]
item = train.nonzero()[1]
train_df = pd.DataFrame(data=np.column_stack((user,item)),columns=['uid','idx'])
train_df = train_df.merge(mapping, 'inner', on= 'idx')

In [146]:
user_test = test.nonzero()[0]
item_test = test.nonzero()[1]
test_df = pd.DataFrame(data=np.column_stack((user_test,item_test)),columns=['uid','idx'])
test_df = test_df.merge(mapping, 'inner', on= 'idx')

In [147]:
train_df.loc[train_df['uid']==28807].shape # 75 rows × 7 columns

(37, 7)

In [148]:
test_df.loc[test_df['uid']==28807].shape # 10 rows × 7 columns

(10, 7)

In [149]:
df_ij[df_ij['uid']== 28807].shape

(47, 2)

In [150]:
import implicit 

model_i = implicit.als.AlternatingLeastSquares(factors = 20, regularization = 0.01, 
                                                         iterations = 100)

In [151]:
# train is user_items sparse matrix, fit(item_users)
model_i.fit(train.T)

100%|██████████| 100.0/100 [02:13<00:00,  1.25s/it]


In [54]:
# Recommends items for a user
recommendations = model_i.recommend(28807, train,  N=10, filter_already_liked_items=True)

In [55]:
recommendations

[(596, 0.4029046),
 (576, 0.38121486),
 (811, 0.28528383),
 (50, 0.2570655),
 (809, 0.25389624),
 (1215, 0.24789166),
 (1159, 0.21242917),
 (1328, 0.21032678),
 (1161, 0.19869144),
 (241, 0.19217595)]

In [56]:
recommendations_df = pd.DataFrame(recommendations, columns=['idx', 'pre'])
rec = recommendations_df.merge(mapping, 'inner', on = 'idx')
rec

,idx,pre,mid,title_en,title_de,title_zh,go_live_at
0,596,0.402905,R18,Blueberry smoothie,Blaubeersmoothie,蓝莓思慕雪,2014-09-27 02:30:00+00:00
1,576,0.381215,R253,Sunset smoothie,Sonnenuntergang Smoothie,落日思慕雪,2015-06-22 03:00:00+00:00
2,811,0.285284,R241,Glowing red smoothie,Sommermorgen-Smoothie,闪亮红色思慕雪,2015-05-24 22:00:00+00:00
3,50,0.257066,R822,One-pot spinach pasta,One-Pot-Pasta mit Spinat,单锅菠菜意面,2017-10-06 02:00:00+00:00
4,809,0.253896,R234,Superfood green smoothie,Grashüpfer Smoothie,超级绿色思慕雪,2015-05-11 04:00:00+00:00
5,1215,0.247892,R394,Colorful acai breakfast bowl,Acai Bowl,缤纷巴西莓早餐,2016-02-24 03:00:00+00:00
6,1159,0.212429,R280,Asian-inspired lemonade,Asiatisch inspirierte Limonade,亚洲风味柠檬水,2015-08-16 03:00:00+00:00
7,1328,0.210327,R325,"Banana, mango, and chia smoothie",Bananen-Mango-Smoothie mit Chiasamen,香蕉芒果奇亚籽思慕雪,2015-10-18 03:00:00+00:00
8,1161,0.198691,R239,Clear your mind smoothie,Klarer Kopf Smoothie,提神思慕雪,2015-05-21 22:00:00+00:00
9,241,0.192176,R507,Cheese and spinach triangles,Käse-Spinat-Dreiecke,菠菜奶酪三角包,2016-03-17 01:00:00+00:00


In [57]:
test_df.loc[test_df['uid']==28807]

,uid,idx,mid,title_en,title_de,title_zh,go_live_at
12471,28807,697,R244,Creamy pasta with broccoli,Cremige Brokkolinudeln,西兰花意大利面,2015-06-01 03:00:00+00:00
17208,28807,474,R272,Strawberry and blackberry popsicles,Erdbeer- und Brombeereis am Stiel,香甜的浆果冰棒,2015-07-20 03:00:00+00:00
22069,28807,133,R1007,Vegetarian tacos with lime-yogurt dip,Frische Tacos mit cremigem Joghurt-Dip,新鲜玉米饼佐奶油酸奶酱,2017-09-20 02:00:00+00:00
22382,28807,750,R1231,Creamy cucumber salad with radishes and croutons,Cremiger Gurkensalat mit Radieschen und Croûtons,黄瓜樱桃萝卜酸奶沙拉,2018-04-18 02:00:00+00:00
29449,28807,446,R297,Raspberry molten chocolate cupcakes,Himbeer-Schokoladen-Cupcakes mit flüssigem Kern,覆盆子熔岩巧克力蛋糕,2015-09-13 03:00:00+00:00
31550,28807,1055,R224,Poached chicken and avocado sandwich,Sandwich mit pochiertem Hähnchen und Avocado,鸡胸肉鳄梨三明治,2015-04-17 03:00:00+00:00
32825,28807,127,R892,White chocolate and berry popsicles,Beereneis am Stiel mit weißer Schokolade,白巧克力浆果棒冰,2017-08-29 02:00:00+00:00
38085,28807,918,R182,Aromatic roasted vegetables,Aromatisches Ofengemüse,烤蔬菜,2014-12-01 12:12:00+00:00
38404,28807,671,R1095,Paloma,Paloma,鸽子鸡尾酒,2018-03-16 02:00:00+00:00
39015,28807,589,R728,Low-carb rice pudding,Low Carb Milchreis,低卡米布丁,2017-05-02 02:00:00+00:00


In [58]:
train_df.loc[train_df['uid']==28807]

,uid,idx,mid,title_en,title_de,title_zh,go_live_at
97105,28807,29,R755,Fettuccine Alfredo,Fettuccine Alfredo,阿弗雷多奶酪意面,2017-07-25 02:00:00+00:00
215877,28807,72,R1037,Goat cheese crème brûlée with lamb's lettuce a...,Ziegenkäse-Crème-Brulée und Feldsalat mit Apfe...,山羊芝士焦糖蛋奶冻佐野苣和苹果酱,2017-11-21 02:00:00+00:00
251538,28807,83,R489,Four-cheese and tomato baked penne,Nudelauflauf Quattro Formaggi,四种奶酪焗意面,2017-04-05 03:00:00+00:00
285754,28807,98,R1006,Black bean and rice veggie burritos,Würzige Burritos mit Avocado,墨西哥牛油果辣卷饼,2017-09-26 02:00:00+00:00
443975,28807,178,C112,Italian-style pasta salad,Italienischer Nudelsalat,意面沙拉,2018-05-08 02:00:00+00:00
498392,28807,202,R699,Pasta with creamy avocado-lime sauce,Spaghetti mit cremiger Avocado-Limetten-Sauce,酸爽青酱意面,2017-04-13 03:00:00+00:00
538244,28807,220,R144,Simple lemon poppy seed muffins,Einfache Zitronen-Mohn-Muffins,简易柠檬罂粟籽玛芬,2014-10-07 03:00:00+00:00
542049,28807,221,R268,Lemon cake with blueberries,Zitronen-Blaubeer-Kuchen,柠檬蓝莓蛋糕,2015-08-30 03:00:00+00:00
592138,28807,235,R134,Pasta verde,Pasta Verde,青酱意面,2014-08-29 08:38:00+00:00
596622,28807,236,R522,Potato soup with horseradish,Kartoffelsuppe mit Meerrettich,辣根土豆汤,2016-11-22 03:00:00+00:00


In [59]:
test.getrow(28807).indices

array([ 127,  133,  446,  474,  589,  671,  697,  750,  918, 1055],
      dtype=int32)

In [60]:
rec[rec.columns[0]].tolist()

[596, 576, 811, 50, 809, 1215, 1159, 1328, 1161, 241]

In [61]:
set(rec[rec.columns[0]].tolist()) & set(test.getrow(12611).indices)

set()

In [71]:
# Calculates a list of similar items
related = model_i.similar_items(459, N=10) # 拍黄瓜

In [72]:
[related[i][0] for i in range(len(related))]

[459, 590, 537, 152, 227, 5, 931, 311, 452, 1032]

In [73]:
related_df = pd.DataFrame(related, columns=['idx', 'pre'])
similar_item = related_df.merge(mapping, 'left', on = 'idx')
similar_item

,idx,pre,mid,title_en,title_de,title_zh,go_live_at
0,459,0.374478,R396,Stuffed mushrooms,Gefüllte Champignons,香烤菠菜奶酪夹馅口蘑,2016-03-02 03:00:00+00:00
1,590,0.307748,R515,Pomegranate and couscous stuffed eggplant,Gefüllte Auberginen,酿茄子,2016-11-03 01:30:00+00:00
2,537,0.289710,C111,Green asparagus tart,Grüne Spargeltarte,绿芦笋酥皮饼,2018-05-07 02:00:00+00:00
3,152,0.287724,R513,Mini eggplant pizzas,Mini-Auberginen-Pizzas,茄子披萨,2016-11-04 02:00:00+00:00
4,227,0.283542,C99,Smoked salmon and spinach rolls,Spinatrollen mit Lachs,烟熏三文鱼菠菜卷,2018-04-16 02:00:00+00:00
5,5,0.282569,R121,Oven ratatouille with feta,Ratatouille mit Feta aus dem Ofen,烤什锦蔬菜配菲达奶酪,2015-08-24 04:00:00+00:00
6,931,0.267536,R530,Roasted vegetables with mustard vinaigrette,Ofengemüse mit Senfsoße,烤蔬菜佐芥末油醋汁,2016-11-24 03:15:00+00:00
7,311,0.266098,C10,Potato and fennel casserole,Fenchel-Kartoffel-Auflauf,土豆茴香焗菜,2017-07-03 02:00:00+00:00
8,452,0.263683,R408,Cauliflower crust pizza with smoked salmon,Low Carb Pizza aus Blumenkohl mit geräuchertem...,花椰菜脆皮披萨配三文鱼,2016-04-01 03:00:00+00:00
9,1032,0.256243,R756,Broccoli and Gorgonzola pizza,Gorgonzola-Brokkoli-Pizza,西兰花蓝纹奶酪披萨,2017-06-20 02:00:00+00:00


In [82]:
def precision_at_k_ran(test, user_items, k=5, user_index=None):
    if not user_index:
        user_index = range(user_items.shape[0])
    item_index = test.nonzero()[1]
    precisions = []
    for user in user_index:
        np.random.seed(1234)
        random_k = np.random.choice(item_index, replace=False,size=k).tolist()
        labels = test.getrow(user).indices
        precision = float(len(set(random_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)

In [62]:
##Here Recall@10 and Precision@10 is the same, since total # of relevant items = # of recommended items = 10
def precision_at_k(model,test, user_items, k=5, user_index=None):
    if not user_index:
        user_index = range(user_items.shape[0])
    user_items = user_items.tocsr()
    precisions = []
    for user in user_index:
        rec = model.recommend(user, user_items,  N=k, filter_already_liked_items= True)
        top_k = [rec[i][0] for i in range(len(rec))]
        labels = test.getrow(user).indices
        precision = float(len(set(top_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)


In [152]:
precision_at_k_ran(test,train,10, user_index)

0.010883002207505519

In [153]:
precision_at_k(model = model_i, test = test, user_items = train, k = 10, user_index = user_index) #regulazation = 0.01


0.05995584988962473

In [154]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors = 20, learning_rate = 0.01, regularization = 0.01, 
                                                         iterations = 100)

In [155]:
model_bpr.fit(train.T)

100%|██████████| 100/100 [01:58<00:00,  1.18s/it, correct=86.40%, skipped=12.40%]


In [84]:
# precision_at_k(model_bpr, test, train, 10, user_index)


0.09984547461368654

In [156]:
bpr_seq = precision_at_k(model_bpr, test, train, 10, user_index)

In [157]:
bpr_seq

0.05527593818984547

In [99]:
recommendations_bpr = model_bpr.recommend(8500, train,  N=10, filter_already_liked_items=True)

In [100]:
recommendations_bpr_df = pd.DataFrame(recommendations_bpr, columns=['idx', 'pre'])

In [101]:
rec_bpr = recommendations_bpr_df.merge(mapping, 'inner', on = 'idx')

In [102]:
rec_bpr

,idx,pre,mid,title_en,title_de,title_zh,go_live_at
0,863,0.770140,R212,Whole braised fish,Geschmorter ganzer Fisch,红烧鲈鱼,2015-11-22 04:30:00+00:00
1,253,0.757684,R250,Buttermilk chicken wings,Buttermilch Chicken Wings,酪乳鸡翅,2015-06-15 03:00:00+00:00
2,1293,0.728589,R2,Angel food cake with fresh strawberries,Angel Food Cake an frischen Erdbeeren,天使蛋糕配新鲜草莓,2014-07-15 12:13:00+00:00
3,1057,0.692897,R239,Clear your mind smoothie,Klarer Kopf Smoothie,提神思慕雪,2015-05-21 22:00:00+00:00
4,217,0.677133,R41,Flourless chocolate cake,Schokoladenkuchen ohne Mehl,零面粉巧克力蛋糕,2014-07-15 12:13:00+00:00
5,398,0.674098,R159,Eggs Benedict,Eier Benedict,班尼迪克蛋,2015-01-25 22:00:00+00:00
6,1189,0.673463,R282,Brazilian-inspired Feijoada,Brasilianisch inspirierte Feijoada,巴西风味黑豆饭,2015-08-14 03:00:00+00:00
7,1316,0.645062,R202,Soy vanilla pudding with blueberry sauce,Soja-Vanillepudding mit Blaubeersoße,蓝莓酱豆奶香草布丁,2015-06-01 03:00:00+00:00
8,328,0.644302,R241,Glowing red smoothie,Sommermorgen-Smoothie,闪亮红色思慕雪,2015-05-24 22:00:00+00:00
9,902,0.638188,R138,Spinach and ricotta ravioli,Spinat-Ricotta-Ravioli,意式菠菜奶酪饺,2014-09-18 22:00:00+00:00


In [86]:
precisions_bpr = []
for user in user_index:
    rec_items = model_bpr.recommend(user, train, 10, filter_already_liked_items=True)
    top_k_bpr = [rec_items[i][0] for i in range(len(rec_items))]
    labels_bpr = test.getrow(user).indices
    precision = float(len(set(top_k_bpr) & set(labels_bpr))) / float(10)
    precision_bpr = float(len(set(top_k_bpr) & set(labels_bpr))) / float(10)
    precisions_bpr.append(precision_bpr)
    print(user,top_k_bpr, labels_bpr,  precision)
np.mean(precisions_bpr)

28807 [622, 1328, 1206, 718, 791, 1281, 1215, 1134, 547, 809] [ 127  133  446  474  589  671  697  750  918 1055] 0.0
43059 [873, 570, 713, 1039, 46, 24, 725, 1465, 863, 1092] [   3   46  356  458  482  570  713  873  955 1129] 0.4
7086 [920, 265, 166, 171, 179, 806, 630, 1316, 851, 892] [  17   59   75  105  188  295  435  585 1034 1381] 0.0
35572 [460, 26, 62, 92, 265, 1219, 179, 150, 467, 391] [  24   75  182  364  605  881  901  903 1146 1162] 0.0
37257 [159, 570, 208, 408, 83, 395, 226, 255, 99, 602] [  99  176  208  233  305  353  379  566 1074 1349] 0.2
28018 [1332, 1094, 630, 920, 166, 46, 234, 1211, 608, 173] [  80  163  353  704  839  868  871 1036 1177 1332] 0.1
11953 [911, 1118, 865, 1207, 1074, 426, 580, 1087, 1004, 1446] [  14   28   79  173  202  484  585  730 1031 1330] 0.0
6154 [1014, 944, 894, 96, 943, 1389, 116, 539, 861, 1443] [ 141  213  416  439  811 1209 1235 1389 1399 1418] 0.1
16156 [1215, 1144, 1075, 1137, 809, 1488, 1199, 1208, 1059, 1328] [ 464  498  654  71

7171 [179, 25, 77, 451, 1077, 229, 225, 80, 777, 723] [  35  194  229  329  454  524  593  608  671 1054] 0.1
43996 [938, 1159, 1254, 492, 1001, 1231, 488, 1064, 1176, 1301] [ 313  604  672  719  814 1057 1060 1146 1215 1279] 0.0
16426 [766, 977, 595, 1202, 493, 352, 510, 1440, 1250, 1343] [  28   94  122  130  478  503  772  890 1025 1140] 0.0
13155 [202, 1110, 979, 26, 684, 503, 392, 681, 542, 304] [  24   28   50   83   99  194  772  867 1120 1332] 0.0
6159 [923, 1081, 121, 713, 98, 1352, 1412, 144, 99, 225] [  24   99  255  310  485  537  608  911 1122 1427] 0.1
45187 [134, 283, 172, 185, 505, 479, 1303, 412, 463, 991] [  64  159  263  408  492  506  629  879 1144 1147] 0.0
1075 [1316, 30, 1092, 688, 1182, 1037, 1203, 726, 1255, 469] [  95  353  443  476  746 1044 1074 1149 1188 1198] 0.0
8407 [1456, 775, 1481, 1403, 1382, 87, 912, 74, 600, 1451] [ 179  197  475  590  835  856 1102 1115 1223 1464] 0.0
1050 [917, 224, 39, 796, 460, 1316, 166, 222, 682, 785] [ 119  420  460  480  619

3272 [87, 260, 515, 432, 427, 984, 410, 659, 1102, 175] [  2 151 183 239 252 302 398 515 516 838] 0.1
34519 [134, 646, 364, 635, 1031, 192, 602, 235, 587, 463] [  24  165  192  207  379  645  757  837  991 1288] 0.1
33175 [393, 554, 384, 819, 356, 594, 75, 1204, 1215, 541] [  70   92  199  203  226  393  435  773 1060 1265] 0.1
6600 [428, 669, 316, 142, 203, 248, 715, 372, 207, 15] [  30   59   80  153  215  242  316  580  733 1206] 0.1
22454 [19, 1426, 398, 120, 712, 550, 69, 5, 815, 822] [  17   41   46  130  176  275  771  808  913 1232] 0.0
39923 [545, 819, 1060, 796, 696, 872, 813, 682, 265, 648] [ 64 222 235 297 300 400 662 792 868 920] 0.0
16105 [39, 682, 733, 24, 613, 156, 46, 92, 904, 265] [ 322  605  608  613  648  657  662  683  720 1040] 0.1
28733 [882, 669, 106, 139, 950, 864, 952, 481, 117, 32] [  37   85  527  705  864  884 1146 1162 1297 1331] 0.1
36252 [596, 576, 810, 1328, 811, 587, 1075, 450, 718, 1204] [ 83 150 161 240 446 450 456 463 793 869] 0.1
25836 [904, 1121, 

14620 [1008, 862, 512, 583, 107, 535, 404, 348, 857, 754] [  51  180  370  502  535  543  649  695 1028 1145] 0.1
7440 [830, 409, 1399, 984, 861, 828, 970, 140, 1101, 765] [ 55  79 301 387 518 656 669 715 749 923] 0.0
17825 [11, 797, 1433, 229, 662, 83, 923, 911, 529, 353] [  11   83  235  265  662  920 1087 1365 1385 1484] 0.3
42217 [600, 912, 1046, 1456, 243, 834, 396, 417, 915, 816] [  15   38   39   74  243  550  602  637  930 1017] 0.1
6725 [41, 333, 159, 267, 83, 448, 275, 13, 269, 29] [  25  141  362  363  448  719 1120 1234 1390 1472] 0.1
44002 [250, 89, 154, 60, 92, 825, 17, 155, 260, 578] [  28  113  174  204  670  690  692 1084 1226 1366] 0.0
12280 [756, 947, 481, 404, 778, 327, 857, 90, 348, 673] [   1   31   61  426  625  903  955 1004 1052 1069] 0.0
12912 [1155, 479, 75, 163, 1154, 1072, 283, 991, 228, 624] [  46   75  204  226  991 1057 1061 1100 1143 1154] 0.3
11570 [886, 870, 1349, 899, 1340, 867, 1361, 1140, 1192, 1198] [   4  218  541  697  761  869 1087 1110 1215 12

34681 [1159, 479, 619, 1151, 1155, 1143, 620, 1032, 1271, 1062] [  75  221  437  455  587  697  868 1139 1159 1331] 0.1
30602 [524, 532, 924, 629, 1436, 800, 254, 238, 1228, 536] [  83  111  183  187  494  532  646  667  675 1122] 0.1
34011 [1159, 1065, 947, 263, 968, 901, 395, 368, 1041, 1007] [ 100  199  275  505  537  767  850 1079 1118 1177] 0.0
38058 [1264, 30, 252, 111, 1255, 475, 1148, 1253, 428, 300] [   9   20   30  141  191  208  333  814  929 1164] 0.1
31126 [426, 1135, 1133, 1063, 1074, 1198, 1004, 275, 1087, 1118] [  24   39   44  150  226  682  906 1031 1040 1074] 0.1
15455 [585, 59, 11, 799, 1073, 580, 911, 485, 194, 1029] [214 235 585 682 700 712 720 772 847 870] 0.1
11097 [1135, 865, 1139, 1298, 1168, 1198, 79, 1004, 1023, 921] [  39   51   61  156  182  277  838  870 1053 1261] 0.0
21459 [1159, 492, 1222, 889, 808, 906, 908, 446, 1164, 1036] [  11  185  458  485  492  808 1155 1159 1303 1312] 0.3
6442 [355, 50, 333, 13, 206, 445, 346, 956, 1, 131] [ 49 114 153 175 218

...

7480 [286, 926, 742, 975, 1090, 979, 830, 1406, 574, 384] [   9  187  217  355  411  652  810 1159 1269 1357] 0.0
1086 [847, 224, 728, 614, 1383, 892, 851, 1466, 445, 899] [  36  151  729  798  851  913  919 1059 1088 1406] 0.1
21280 [1036, 446, 1216, 161, 1146, 814, 868, 1164, 889, 1162] [ 161  283  446  479  587  621  811 1036 1216 1256] 0.4
10922 [15, 659, 789, 524, 119, 641, 153, 238, 542, 710] [  78  234  292  370  460  503  624  629  772 1154] 0.0
13324 [1183, 613, 646, 1466, 412, 1041, 450, 426, 661, 156] [ 134  190  266  450  621  699  723  803  860 1057] 0.1
5614 [1098, 900, 1460, 1295, 1180, 806, 489, 507, 936, 726] [ 241  547  637  645 1021 1059 1113 1180 1384 1394] 0.1
33230 [119, 494, 320, 75, 214, 319, 527, 444, 1223, 12] [  80  166  238  265  485  564  662 1125 1207 1223] 0.1
1804 [1020, 45, 16, 1259, 455, 22, 1314, 10, 111, 193] [   6   10  101  179  268  645 1085 1209 1247 1272] 0.1
10618 [580, 911, 80, 860, 1446, 172, 148, 194, 543, 1004] [  11  182  202  216  230  46

21699 [410, 1109, 223, 52, 151, 649, 71, 35, 260, 60] [  36   56   59   64  649  653  803  884 1181 1281] 0.1
12010 [1240, 1178, 1191, 1179, 1384, 1181, 1195, 590, 972, 1420] [ 109  263  308  330  426  674 1274 1275 1276 1298] 0.0
5542 [311, 123, 122, 665, 130, 847, 369, 289, 165, 753] [ 274  281  352  364  402  712 1196 1213 1289 1472] 0.0
1403 [855, 50, 180, 858, 322, 442, 940, 698, 267, 423] [  29   42   50   80   92  353  463  551  706 1077] 0.1
20425 [20, 236, 579, 1489, 610, 494, 860, 958, 214, 33] [  20   80  190  233  259  263  317  337 1110 1226] 0.1
24047 [527, 1227, 254, 709, 78, 152, 71, 579, 1323, 532] [  29   78   83  125  204  532  544  605  636 1424] 0.2
21433 [319, 343, 735, 1446, 132, 233, 527, 1385, 320, 535] [  20   28   33   80  148  178  239  265  514 1206] 0.0
44663 [89, 1262, 712, 448, 386, 1041, 1449, 154, 87, 74] [  10   89  122  210  229  620  663 1058 1096 1129] 0.1
11685 [125, 670, 853, 231, 981, 722, 2, 409, 8, 850] [  59  171  195  321  402  570  585  700

38113 [1029, 803, 235, 226, 494, 863, 529, 46, 602, 202] [  14   75  113  226  431  492  767  803  991 1162] 0.2
42972 [1229, 720, 1451, 515, 690, 716, 381, 105, 402, 9] [  56   92  165  277  453  485  641  716 1226 1305] 0.1
27308 [149, 674, 131, 3, 181, 345, 22, 289, 768, 902] [  98  289  360  450  771  895 1074 1219 1314 1346] 0.1
7651 [57, 670, 5, 165, 424, 164, 649, 722, 10, 1003] [  24  179  221  379  395  428  515  580  793 1048] 0.0
35496 [1081, 141, 121, 376, 844, 138, 713, 1435, 84, 71] [  85  121  138  161  214  227  492  713 1088 1174] 0.3
37141 [630, 1192, 373, 634, 921, 1409, 1332, 867, 865, 1345] [  33  419  516  577  578  614  806  920 1138 1321] 0.0
17769 [735, 224, 580, 847, 300, 1397, 1109, 594, 31, 769] [  0  91 208 223 455 485 494 655 720 903] 0.0
17068 [25, 29, 454, 993, 1294, 1247, 1037, 1085, 1196, 467] [  21  275  454  521  772  866  993 1104 1243 1258] 0.2
45021 [1036, 277, 283, 1022, 717, 1216, 226, 494, 619, 479] [  21  277  450  479  494  697  717 1036 1158

43501 [32, 571, 735, 775, 84, 35, 31, 912, 530, 128] [ 153  230  387  925  968 1062 1072 1082 1135 1207] 0.0
3218 [75, 543, 723, 1106, 59, 121, 98, 277, 1081, 821] [  36   38   64   77  119  135  176  320  641 1125] 0.0
11654 [805, 812, 810, 1204, 1137, 1144, 1452, 811, 658, 1075] [ 307  576  708  730  805  810  811  916 1266 1452] 0.4
39738 [656, 2, 531, 1406, 950, 139, 874, 669, 748, 953] [  31  126  243  268  270  387  450  782  980 1011] 0.0
40189 [17, 825, 144, 281, 138, 250, 35, 737, 244, 1435] [  31  121  272  277  373  376  576  713  734 1274] 0.0
35218 [1125, 1442, 566, 1415, 546, 1058, 1257, 85, 1147, 1052] [ 106  550  557  719  869  870  909  942 1112 1138] 0.0
31728 [1143, 793, 866, 1100, 697, 283, 11, 1155, 226, 1028] [ 283  490  491  579  585  866  991 1047 1143 1162] 0.3
16548 [1442, 106, 603, 779, 1147, 552, 1224, 1285, 1425, 601] [ 219  353  532  579  936 1223 1292 1361 1376 1484] 0.0
42039 [1370, 194, 32, 580, 214, 33, 230, 1309, 798, 163] [  23  508  531  560  566  9

0.09984547461368654

In [73]:
len(precisions_bpr)

4530

In [ ]:
len(user_index)

In [108]:
precision_at_k(model_bpr, train, test, 10, user_index)

0.1311046511627907

In [5]:
ds = pd.read_csv('/Users/xun/Documents/Thesis/ml-1m.test.negative.csv')

In [7]:
ds = pd.read_csv('/Users/xun/Documents/Thesis/ml-1m.test.negative.csv')

